In [1]:
import polars as pl
import numpy as np, pandas as pd
print(
    f"Polars version = {pl.__version__}, Numpy version = {np.__version__}"
)
%load_ext autoreload
%autoreload 2
from src import data_transformation
from src.model import prepare_lgbm_dataset_with_weights, train_or_load_lgbm, evaluate_model

Polars version = 1.32.2, Numpy version = 1.26.4


# 数据处理
<mark>后续可能会进一步封装代码，让这部分更简洁</mark>

In [2]:
from pathlib import Path
INPUTS = Path('./data/inputs')
INTERMEDIATES = Path('./data/intermediates')
OUTPUTS = Path('./data/outputs')

INPUTS.mkdir(parents=True, exist_ok=True)
INTERMEDIATES.mkdir(parents=True, exist_ok=True)
OUTPUTS.mkdir(parents=True, exist_ok=True)

In [3]:
## 
datasets = data_transformation.load_inputs(INPUTS)
intermediate_datasets_pth = {
    "merged_reserve": INTERMEDIATES/"merged_reserve.csv",
    "date_info": INTERMEDIATES/"date_info.csv",
    "merged_store_info": INTERMEDIATES/"merged_store_info.csv",
    "train_visit_data": INTERMEDIATES/"train_visit_data.csv",
    "test_visit_data": INTERMEDIATES/"test_visit_data.csv"
}

if not intermediate_datasets_pth["merged_reserve"].exists():
    merged_reserve = data_transformation.merge_reservation(**datasets)
    merged_reserve.sink_csv(INTERMEDIATES/"merged_reserve.csv")

if not intermediate_datasets_pth["date_info"].exists():
    datasets["date_info"].sink_csv(INTERMEDIATES/"date_info.csv")

if not intermediate_datasets_pth["merged_store_info"].exists():
    datasets["air_store_info"].sink_csv(INTERMEDIATES/"merged_store_info.csv")

if not intermediate_datasets_pth["train_visit_data"].exists():
    datasets["air_visit_data"].filter(pl.col("visit_date") <= pl.lit("2017-03-10")).sink_csv(INTERMEDIATES/"train_visit_data.csv")
if not intermediate_datasets_pth["test_visit_data"].exists():
    datasets["air_visit_data"].filter(pl.col("visit_date") > pl.lit("2017-03-10")).sink_csv(INTERMEDIATES/"test_visit_data.csv")

In [4]:
output_datasets_pth = {
    "hpg_train": OUTPUTS/"hpg_train.parquet",
    "hpg_test": OUTPUTS/"hpg_test.parquet",
    "air_train": OUTPUTS/"air_train.parquet",
    "air_test": OUTPUTS/"air_test.parquet",
}

if any([not pth.exists() for pth in output_datasets_pth.values()]):
    train, test = data_transformation.add_features_pipeline(
        intermediate_datasets_pth["train_visit_data"],
        intermediate_datasets_pth["test_visit_data"],
        intermediate_datasets_pth["merged_reserve"],
        intermediate_datasets_pth["date_info"],
        intermediate_datasets_pth["merged_store_info"]
    )
    hpg_list = pd.read_parquet(INPUTS/"store_id_relation.parquet")["air_store_id"].tolist()

    train.query("air_store_id in @hpg_list").to_parquet(output_datasets_pth["hpg_train"])
    train.query("air_store_id not in @hpg_list").to_parquet(output_datasets_pth["air_train"])
    test.query("air_store_id in @hpg_list").to_parquet(output_datasets_pth["hpg_test"])
    test.query("air_store_id not in @hpg_list").to_parquet(output_datasets_pth["air_test"])

# 模型训练

In [5]:
# 源域
air_train = pd.read_parquet(OUTPUTS/"air_train.parquet")

# 目标域
hpg_train = pd.read_parquet(OUTPUTS/"hpg_train.parquet")
hpg_test = pd.read_parquet(OUTPUTS/"hpg_test.parquet")

x_cols = hpg_train.drop(columns = ["air_store_id","visit_date","visitors"]).columns
y_cols = ["visitors"]

## Baseline
baseline 的 `RMSLE` = 0.5664

In [16]:
# Baseline Model

# 1. 准备数据（自动从训练数据中划分训练集和验证集）
train_matrix, valid_matrix, te_x, te_y = prepare_lgbm_dataset_with_weights(
    train_data=hpg_train,    # 完整的训练数据
    test_data=hpg_test,      # 测试数据
    x_cols=x_cols,
    y_cols=y_cols,
    weight_col='day_gap',
    valid_days=7             # 使用最近7天作为验证集
)

# 2. 训练或加载模型
model = train_or_load_lgbm(
    train_matrix=train_matrix,
    valid_matrix=valid_matrix,    # 使用验证集进行早停
    model_path='./lgbm_weights/hpg_model.pkl'
)

# 3. 评估模型
results = evaluate_model(model, te_x, te_y)
print(f"RMSLE Score: {results['rmsle']:.4f}")

Train data date range: 2016-01-16 to 2017-03-03
Valid data date range: 2017-03-04 to 2017-03-10
Train samples: 38841, Valid samples: 919
Training model with early stopping...
Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[1103]	valid_0's rmse: 0.549814
Model saved to ./lgbm_weights/hpg_model.pkl
RMSLE Score: 0.5664


## 迁移学习部分

简单的样本迁移（用`air_train`训练模型，然后直接预测）<br>
`RMSLE` = 0.5603(<span style="color:green">-0.0063</span> vs baseline)

In [14]:
## 
train_matrix, valid_matrix, te_x, te_y = prepare_lgbm_dataset_with_weights(
    train_data=air_train,    # 样本迁移 air 的训练数据
    test_data=hpg_test,      # 测试数据
    x_cols=x_cols,
    y_cols=y_cols,
    weight_col='day_gap',
    valid_days=7            
)

# 2. 训练或加载模型
model = train_or_load_lgbm(
    train_matrix=train_matrix,
    valid_matrix=valid_matrix,    # 使用验证集进行早停
    model_path='./lgbm_weights/air_model.pkl'
)

# 3. 评估模型
results = evaluate_model(model, te_x, te_y)
print(f"RMSLE Score: {results['rmsle']:.4f}")

Train data date range: 2016-01-16 to 2017-03-03
Valid data date range: 2017-03-04 to 2017-03-10
Train samples: 174300, Valid samples: 4079
Loading model from ./lgbm_weights/air_model.pkl
RMSLE Score: 0.5603


如果进行微调呢？<br>
`RMSLE` = 0.5673(<span style="color:red">+0.0009</span> vs baseline)

In [12]:
# 迁移学习：源域预训练 -> 目标域微调
from src.model.transfer import finetune_from_source
FINETUNED_MODEL_PATH = Path("./lgbm_weights/air_to_hpg_finetuned.pkl")

finetune_result = finetune_from_source(
    train_data=hpg_train,
    test_data=hpg_test,
    x_cols=x_cols,
    y_cols=y_cols,
    source_model_path="./lgbm_weights/air_model.pkl",  # 先运行前面的源域训练单元生成
    finetuned_model_path=FINETUNED_MODEL_PATH,
    weight_col="day_gap",
    valid_days=7,
    learning_rate=1e-3,
    num_round=2000,
)


print("="*20)
print(f"Finetune RMSLE: {finetune_result['rmsle']:.4f}")
FINETUNED_MODEL_PATH.unlink()

Train data date range: 2016-01-16 to 2017-03-03
Valid data date range: 2017-03-04 to 2017-03-10
Train samples: 38841, Valid samples: 919
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[4936]	valid_0's rmse: 0.55974
Finetune RMSLE: 0.5620


如果进行特征迁移呢？<br>
`RMSLE` = 0.5913(<span style="color:red">+0.0249</span> vs baseline)

In [ ]:
# 迁移学习：特征迁移（CORAL）
from src.model.transfer import coral_align_source_to_target

# 1) 将源域特征对齐到目标域分布
coral_air_train = coral_align_source_to_target(
    source_df=air_train,
    target_df=hpg_train,
    feature_cols=x_cols,
)

# 2) 使用对齐后的源域数据训练，在目标域测试集评估
coral_train_matrix, coral_valid_matrix, coral_te_x, coral_te_y = prepare_lgbm_dataset_with_weights(
    train_data=coral_air_train,
    test_data=hpg_test,
    x_cols=x_cols,
    y_cols=y_cols,
    weight_col='day_gap',
    valid_days=7,
)

coral_model = train_or_load_lgbm(
    train_matrix=coral_train_matrix,
    valid_matrix=coral_valid_matrix,
    model_path=None,
)

coral_results = evaluate_model(coral_model, coral_te_x, coral_te_y)
print("="*20)
print(f"CORAL RMSLE: {coral_results['rmsle']:.4f}")

Train data date range: 2016-01-16 to 2017-03-03
Valid data date range: 2017-03-04 to 2017-03-10
Train samples: 174300, Valid samples: 4079
Training model with early stopping...
Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[3370]	valid_0's rmse: 0.469629
CORAL RMSLE: 0.5913
